In [1]:
import os
import gzip
import subprocess
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def get_df(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [3]:
DATASET = 'Office_Products'
RAW_PATH = os.path.join('./', DATASET)
DATA_FILE = 'reviews_{}_5.json.gz'.format(DATASET)
META_FILE = 'meta_{}.json.gz'.format(DATASET)

RANDOM_SEED = 0
NEG_ITEMS = 99

# Load Data

1. Load interaction data and item metadata
2. Filter out unuseful items in metadata
3. Calculate basic statistics

In [4]:
# download data if not exists

if not os.path.exists(RAW_PATH):
    subprocess.call('mkdir ' + RAW_PATH, shell=True)
if not os.path.exists(os.path.join(RAW_PATH, DATA_FILE)):
    print('Downloading interaction data into ' + RAW_PATH)
    subprocess.call(
        'cd {} && curl -O http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_{}_5.json.gz'
        .format(RAW_PATH, DATASET), shell=True)
if not os.path.exists(os.path.join(RAW_PATH, META_FILE)):
    print('Downloading item metadata into ' + RAW_PATH)
    subprocess.call(
        'cd {} && curl -O http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_{}.json.gz'
        .format(RAW_PATH, DATASET), shell=True)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.6M  100 17.6M    0     0  3329k      0  0:00:05  0:00:05 --:--:-- 4531k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

100 45.3M  100 45.3M    0     0  4951k      0  0:00:09  0:00:09 --:--:-- 7944k


In [5]:
data_df = get_df(os.path.join(RAW_PATH, DATA_FILE))
data_df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5.0,"A solid performer, and long time friend",1094169600,"09 3, 2004"
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5.0,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007"
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2.0,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011"
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5.0,One of the last of an almost extinct species,1145404800,"04 19, 2006"
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5.0,Still the best,1375574400,"08 4, 2013"


In [6]:
meta_df = get_df(os.path.join(RAW_PATH, META_FILE))
meta_df.head()

,asin,description,price,imUrl,related,salesRank,categories,title,brand
0,0078800242,All in one TeacherWorks Plus CD-ROM,93.06,http://ecx.images-amazon.com/images/I/41K1aBkl...,{'buy_after_viewing': ['007861970X']},{'Software': 18529},"[[Office Products, Office & School Supplies, C...",NaN,NaN
1,0113000316,High quality inkjet cartridges use high-densit...,NaN,http://ecx.images-amazon.com/images/I/51AMwP3D...,NaN,NaN,"[[Office Products, Office & School Supplies, P...",123GetInk -14-pack 5-black 3-cyan 3-magenta 3-...,NaN
2,043928631X,"Harry Potter living bookmark showing Harry, He...",NaN,http://ecx.images-amazon.com/images/I/41SulB7T...,NaN,NaN,"[[Office Products, Office & School Supplies, L...",Harry Potter Lenticular Hologram Bookmark - Ha...,NaN
3,0439340039,Windows based computer game.,NaN,http://ecx.images-amazon.com/images/I/51zQE0w%...,NaN,{'Software': 32784},"[[Office Products, Office & School Supplies, E...",NaN,NaN
4,0439394058,"126 pieces: 23"" tall schoolhouse calendar, 12 ...",11.64,http://ecx.images-amazon.com/images/I/51DFp0Lg...,"{'also_bought': ['B000QE1HHU', 'B00207MG4Y', '...",NaN,"[[Office Products, Office & School Supplies, E...",Scholastic SC939405 All-In-One Schoolhouse Cal...,Scholastic


In [7]:
# Only retain items that appear in interaction data

useful_meta_df = meta_df[meta_df['asin'].isin(data_df['asin'])].reset_index(drop=True)
all_items = set(useful_meta_df['asin'].values.tolist())

def related_filter(related_dict):
    out_dict = dict()
    if related_dict is not np.nan:
        for r in related_dict:
            out_dict[r] = list(all_items & set(related_dict[r]))
    return out_dict

useful_meta_df['related'] = useful_meta_df['related'].apply(related_filter)

### Statistics

In [8]:
n_users = data_df['reviewerID'].value_counts().size
n_items = data_df['asin'].value_counts().size
n_clicks = len(data_df)
min_time = data_df['unixReviewTime'].min()
max_time = data_df['unixReviewTime'].max()

In [9]:
time_format = '%Y-%m-%d'

print('# Users:', n_users)
print('# Items:', n_items)
print('# Interactions:', n_clicks)
print('Time Span: {}/{}'.format(
    datetime.utcfromtimestamp(min_time).strftime(time_format),
    datetime.utcfromtimestamp(max_time).strftime(time_format))
)

# Users: 4905
# Items: 2420
# Interactions: 53258
Time Span: 2000-09-29/2014-07-23


# Build Dataset

### Interaction data

In [10]:
np.random.seed(RANDOM_SEED)

In [11]:
out_df = data_df.rename(columns={'asin': 'item_id', 'reviewerID': 'user_id', 'unixReviewTime': 'time'})
out_df = out_df[['user_id', 'item_id', 'time']]
out_df = out_df.drop_duplicates(['user_id', 'item_id', 'time'])
out_df = out_df.sort_values(by=['time', 'user_id'], kind='mergesort').reset_index(drop=True)
out_df.head()

,user_id,item_id,time
0,A12DQZKRKTNF5E,B0000AG93P,970185600
1,A3TB9HNQR54B5V,B00004VVIX,983404800
2,A2DN7RUNX06BZ1,B00004VVIX,996105600
3,A37MH7ICH80QOX,B000059RDW,996278400
4,A3S87ZOPB3UM9N,B00004Z6NA,1007078400


In [12]:
# reindex (start from 1)

# uids = sorted(out_df['user_id'].unique())
uids = out_df['user_id'].unique()
user2id = dict(zip(uids, range(1, len(uids) + 1)))
# iids = sorted(out_df['item_id'].unique())
iids = out_df['item_id'].unique()
item2id = dict(zip(iids, range(1, len(iids) + 1)))

out_df['user_id'] = out_df['user_id'].apply(lambda x: user2id[x])
out_df['item_id'] = out_df['item_id'].apply(lambda x: item2id[x])
out_df.head()

,user_id,item_id,time
0,1,1,970185600
1,2,2,983404800
2,3,2,996105600
3,4,3,996278400
4,5,4,1007078400


In [13]:
# leave one out spliting

clicked_item_set = dict()
for user_id, seq_df in out_df.groupby('user_id'):
    clicked_item_set[user_id] = set(seq_df['item_id'].values.tolist())
    
def generate_dev_test(data_df):
    result_dfs = []
    n_items = data_df['item_id'].value_counts().size
    for idx in range(2):
        result_df = data_df.groupby('user_id').tail(1).copy()
        data_df = data_df.drop(result_df.index)
        neg_items = np.random.randint(1, n_items + 1, (len(result_df), NEG_ITEMS))
        for i, uid in enumerate(result_df['user_id'].values):
            user_clicked = clicked_item_set[uid]
            for j in range(len(neg_items[i])):
                while neg_items[i][j] in user_clicked:
                    neg_items[i][j] = np.random.randint(1, n_items + 1)
        result_df['neg_items'] = neg_items.tolist()
        result_dfs.append(result_df)
    return result_dfs, data_df

In [14]:
leave_df = out_df.groupby('user_id').head(1)
data_df = out_df.drop(leave_df.index)

[test_df, dev_df], data_df = generate_dev_test(data_df)
train_df = pd.concat([leave_df, data_df]).sort_index()

len(train_df), len(dev_df), len(test_df)

(43448, 4905, 4905)

In [15]:
train_df.head()

,user_id,item_id,time
0,1,1,970185600
1,2,2,983404800
2,3,2,996105600
3,4,3,996278400
4,5,4,1007078400


In [16]:
test_df.head()

,user_id,item_id,time,neg_items
35,21,27,1070582400,"[1654, 836, 764, 1732, 1034, 278, 1779, 1829, ..."
322,203,120,1170374400,"[87, 873, 2060, 308, 1105, 2081, 1207, 1153, 2..."
484,334,189,1191024000,"[866, 1955, 1113, 2082, 2281, 768, 2085, 1793,..."
550,71,180,1197763200,"[874, 787, 1315, 438, 1848, 1533, 1021, 942, 4..."
1384,190,402,1240099200,"[1732, 2262, 2151, 2376, 596, 1248, 2305, 1132..."


In [17]:
# save results

train_df.to_csv(os.path.join(RAW_PATH, 'train.csv'), sep='\t', index=False)
dev_df.to_csv(os.path.join(RAW_PATH, 'dev.csv'), sep='\t', index=False)
test_df.to_csv(os.path.join(RAW_PATH, 'test.csv'), sep='\t', index=False)

### Item Metadata

In [18]:
# level-2 category

l2_cate_lst = list()
for cate_lst in useful_meta_df['categories']:
    l2_cate_lst.append(cate_lst[0][2] if len(cate_lst[0]) > 2 else np.nan)
useful_meta_df['l2_category'] = l2_cate_lst  
l2_cates = sorted(useful_meta_df['l2_category'].dropna().unique())
l2_dict = dict(zip(l2_cates, range(1, len(l2_cates) + 1)))
useful_meta_df['l2_category'] = useful_meta_df['l2_category'].apply(lambda x: l2_dict[x] if x == x else 0)

In [19]:
# brand

brand_lst = list()
for brand in useful_meta_df['brand']:
    brand_lst.append(brand if not pd.isnull(brand) else np.nan)
useful_meta_df['l1_brand'] = brand_lst
# brands = sorted(useful_meta_df['brand'].dropna().unique())
brands = useful_meta_df['l1_brand'].dropna().unique()
brand_dict = dict(zip(brands, range(1, len(brands) + 1)))
useful_meta_df['l1_brand'] = useful_meta_df['l1_brand'].apply(lambda x: brand_dict[x] if x == x else 0)

In [20]:
useful_meta_df

,asin,description,price,imUrl,related,salesRank,categories,title,brand,l2_category,l1_brand
0,B00000JBLU,"The classic mini-desktop calculator for home, ...",10.05,http://ecx.images-amazon.com/images/I/51cpdDRB...,"{'also_bought': ['B000MFJNVK', 'B000Y52D5G', '...",{},"[[Office Products, Office Electronics, Calcula...",Texas Instruments TI-1795 SV Standard Function...,Texas Instruments,4,1
1,B00000JBNX,The new two-line TI-30X IIS with the memory re...,8.38,http://ecx.images-amazon.com/images/I/411XXAyD...,"{'also_bought': ['B003155XYO', 'B004F9QBGE', '...",{},"[[Office Products, Office Electronics, Calcula...",Texas Instruments TI-30X IIS 2-Line Scientific...,Texas Instruments,4,1
2,B00000JBO8,Perhaps if we'd had this calculator in high sc...,104.99,http://ecx.images-amazon.com/images/I/51QZKWB0...,"{'also_bought': [], 'bought_together': [], 'bu...",{},"[[Office Products, Office Electronics, Calcula...",Texas Instruments TI-83 Graphing Calculator,Texas Instruments,4,1
3,B00000JBLH,If you bought yourself a financial calculator ...,51.94,http://ecx.images-amazon.com/images/I/415w5D26...,"{'also_bought': ['B00000JZKB', 'B001PLII3E', '...",NaN,"[[Office Products, Office Electronics, Calcula...",HP 12C Financial Calculator,HP,4,2
4,B00000JFNV,"Perfect for invitations, thank you notes, movi...",8.42,http://ecx.images-amazon.com/images/I/51q11PWY...,"{'also_bought': ['B00004Z5QO', 'B0000721Z3', '...",NaN,"[[Office Products, Office & School Supplies, P...",Avery Half-Fold Greeting Cards for Inkjet Prin...,Avery,25,3
...,...,...,...,...,...,...,...,...,...,...,...
2415,B00K5UZP2S,,529.99,http://ecx.images-amazon.com/images/I/41KZwQuZ...,"{'also_bought': ['B000SDY14K', 'B00I4XMEYA'], ...",{},"[[Office Products, Office Electronics, Printer...",Brother Printer MFC-L8850CDW Wireless Color La...,Brother,29,6
2416,B00KLLBEPI,NaN,10.99,http://ecx.images-amazon.com/images/I/41a7pEjw...,"{'also_viewed': ['B004F9QBGE', 'B00DGY5IP4', '...",NaN,"[[Office Products, Office & School Supplies, P...","Classic Glow Filler Paper, Wide Ruled,600, MAD...",NaN,25,0
2417,B00KOO594O,,54.42,http://ecx.images-amazon.com/images/I/41wjtN1j...,"{'also_viewed': ['B004QWZINW'], 'buy_after_vie...",NaN,"[[Office Products, Office & School Supplies, F...","SentrySafe CHW20221 Medium Chest Safe, Charcoa...",NaN,19,0
2418,B00KOO599O,,79.43,http://ecx.images-amazon.com/images/I/51IqUfxT...,"{'also_viewed': ['B000Z8W7I0', 'B002ONB4GA', '...",NaN,"[[Office Products, Office & School Supplies, F...","SentrySafe FHW40220 Large File Safe, Charcoal ...",NaN,19,0


In [21]:
item_meta_data = dict()
for idx in range(len(useful_meta_df)):
    info = useful_meta_df.iloc[idx]['related']
    item_meta_data[idx] = {
        'item_id': item2id[useful_meta_df.iloc[idx]['asin']],
        'i_category': useful_meta_df.iloc[idx]['l2_category'],
        'i_brand': useful_meta_df.iloc[idx]['l1_brand'],
        'r_complement': list(map(lambda x: item2id[x], info['also_bought'])) if 'also_bought' in info else [],
        'r_substitute': list(map(lambda x: item2id[x], info['also_viewed'])) if 'also_viewed' in info else [],
    }

item_meta_df = pd.DataFrame.from_dict(item_meta_data, orient='index')
item_meta_df = item_meta_df[['item_id', 'i_category', 'i_brand', 'r_complement', 'r_substitute']]
item_meta_df

,item_id,i_category,i_brand,r_complement,r_substitute
0,170,4,1,"[1532, 1536, 2156, 1344, 411, 388, 161, 21, 73...",[]
1,25,4,1,"[1295, 1856, 2078, 561, 805, 1759, 21, 635, 65...",[]
2,31,4,1,[],[]
3,6,4,2,"[14, 1112, 478, 114, 434, 50]","[14, 180, 114, 25, 50]"
4,859,25,3,"[157, 12, 1073, 1177, 341, 1142, 293, 1722, 64...","[846, 1854, 1722, 1235, 647, 1101, 1073, 558]"
...,...,...,...,...,...
2415,2399,29,6,"[392, 2358]",[]
2416,2418,25,0,[],"[1856, 2159, 1292, 566, 1016, 1705, 559, 1759,..."
2417,2401,19,0,[],[1698]
2418,2406,19,0,[],"[1260, 681, 2401, 1698]"


In [22]:
# save results

item_meta_df.to_csv(os.path.join(RAW_PATH, 'item_meta.csv'), sep='\t', index=False)

In [23]:
useful_meta_df

,asin,description,price,imUrl,related,salesRank,categories,title,brand,l2_category,l1_brand
0,B00000JBLU,"The classic mini-desktop calculator for home, ...",10.05,http://ecx.images-amazon.com/images/I/51cpdDRB...,"{'also_bought': ['B000MFJNVK', 'B000Y52D5G', '...",{},"[[Office Products, Office Electronics, Calcula...",Texas Instruments TI-1795 SV Standard Function...,Texas Instruments,4,1
1,B00000JBNX,The new two-line TI-30X IIS with the memory re...,8.38,http://ecx.images-amazon.com/images/I/411XXAyD...,"{'also_bought': ['B003155XYO', 'B004F9QBGE', '...",{},"[[Office Products, Office Electronics, Calcula...",Texas Instruments TI-30X IIS 2-Line Scientific...,Texas Instruments,4,1
2,B00000JBO8,Perhaps if we'd had this calculator in high sc...,104.99,http://ecx.images-amazon.com/images/I/51QZKWB0...,"{'also_bought': [], 'bought_together': [], 'bu...",{},"[[Office Products, Office Electronics, Calcula...",Texas Instruments TI-83 Graphing Calculator,Texas Instruments,4,1
3,B00000JBLH,If you bought yourself a financial calculator ...,51.94,http://ecx.images-amazon.com/images/I/415w5D26...,"{'also_bought': ['B00000JZKB', 'B001PLII3E', '...",NaN,"[[Office Products, Office Electronics, Calcula...",HP 12C Financial Calculator,HP,4,2
4,B00000JFNV,"Perfect for invitations, thank you notes, movi...",8.42,http://ecx.images-amazon.com/images/I/51q11PWY...,"{'also_bought': ['B00004Z5QO', 'B0000721Z3', '...",NaN,"[[Office Products, Office & School Supplies, P...",Avery Half-Fold Greeting Cards for Inkjet Prin...,Avery,25,3
...,...,...,...,...,...,...,...,...,...,...,...
2415,B00K5UZP2S,,529.99,http://ecx.images-amazon.com/images/I/41KZwQuZ...,"{'also_bought': ['B000SDY14K', 'B00I4XMEYA'], ...",{},"[[Office Products, Office Electronics, Printer...",Brother Printer MFC-L8850CDW Wireless Color La...,Brother,29,6
2416,B00KLLBEPI,NaN,10.99,http://ecx.images-amazon.com/images/I/41a7pEjw...,"{'also_viewed': ['B004F9QBGE', 'B00DGY5IP4', '...",NaN,"[[Office Products, Office & School Supplies, P...","Classic Glow Filler Paper, Wide Ruled,600, MAD...",NaN,25,0
2417,B00KOO594O,,54.42,http://ecx.images-amazon.com/images/I/41wjtN1j...,"{'also_viewed': ['B004QWZINW'], 'buy_after_vie...",NaN,"[[Office Products, Office & School Supplies, F...","SentrySafe CHW20221 Medium Chest Safe, Charcoa...",NaN,19,0
2418,B00KOO599O,,79.43,http://ecx.images-amazon.com/images/I/51IqUfxT...,"{'also_viewed': ['B000Z8W7I0', 'B002ONB4GA', '...",NaN,"[[Office Products, Office & School Supplies, F...","SentrySafe FHW40220 Large File Safe, Charcoal ...",NaN,19,0


In [24]:
import html
import re
from tqdm import tqdm
def clean_text(text):
    text = html.unescape(text)
    text = re.sub(r'["\n\r]*', '', text)
    return text
outf = open(os.path.join(RAW_PATH, f'{DATASET}.text'),"w")
outf.write(f"id\ttext\n")
for index, row in tqdm(useful_meta_df.iterrows(),total=len(useful_meta_df)):
    item_id, title, categories, brand = row['asin'], row["title"], row['categories'],row['brand']
    text = ""
    if not pd.isnull(title):
        title = " ".join(title.strip().split()[:32])
        title = clean_text(title)
        text += title
    if len(categories) > 0:
        for cate in categories:
            if cate[0] == "Electronics":
                category = clean_text(cate[-1])
                text += " " + category
    #     item2category[item_id] = category
    if not pd.isnull(brand):
        brand = clean_text(brand)
        text += " " + brand
        # item2brand[item_id] = clean_text(brand)
    outf.write(f"{item_id}\t{text}\n")

100%|██████████| 2420/2420 [00:00<00:00, 9308.73it/s]


In [25]:
def load_unit2index(file):
    unit2index = dict()
    with open(file, 'r') as fp:
        for line in fp:
            unit, index = line.strip().split('\t')
            unit2index[unit] = int(index)
    return unit2index


def write_remap_index(unit2index, file):
    with open(file, 'w') as fp:
        for unit in unit2index:
            fp.write(unit + '\t' + str(unit2index[unit]) + '\n')

write_remap_index(user2id, os.path.join(RAW_PATH, f'{DATASET}.user2index'))
write_remap_index(item2id, os.path.join(RAW_PATH, f'{DATASET}.item2index'))